In [1]:
from pipeline import ClassificationPipeline
import pandas as pd

In [2]:
TEST_PATH = r'C:\Users\User Ambev\Desktop\Célula de analytics\Projetos\iva-classifier\data\external\sample.csv'
PATH = r'C:\Users\User Ambev\Desktop\Célula de analytics\Projetos\iva-classifier\data\external\history v3.csv'
#data = pd.read_csv(PATH)

In [3]:
#data.head()

In [4]:
model_setup = {
    #user defined
    'model_id':'teste1',
    'date_col':'DatadoDocumento',
    'dependent_vars': 'IVAMIRO',
    'cat_features': ['IVAPC','PEP','Filial','Material','UF','TpImposto','UFUnd'],
    'num_features': [],    
    # default
    'train_frac_split':0.8,
    'pd_encoding':'ansi',
    'pd_sep':',',
    'date_col':None,
    'fastai_cycles':12,
    'cat_emb_szs': {
        'IVAPC':10,
        #'PEP':,
        'Filial':10,
        'Material':40,
        'UF': 10,
        #'TpImposto':,
        'UFUnd':10
    },
    'fastai_bs':512
}

In [5]:
model = ClassificationPipeline(**model_setup)

In [8]:
data = pd.read_csv(PATH)
test_data = pd.read_csv(TEST_PATH)

C:\Anaconda3\lib\site-packages\IPython\core\interactiveshell.py:3063: DtypeWarning: Columns (0,1,3,7,9,13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [7]:
model.fit(data = data)

dropping NaNs...
Splitting data...
train size: 64.0%
validation size: 16.0%
test size: 20.0%
Dropped classes: ['IVAMIRO']
100.0% of validation set kept


IVAPC,PEP,Filial,Material,UF,TpImposto,UFUnd,target
QS,#na#,BR3R,39044,RS,B1,RS,QS
I7,U,BR45,#na#,PR,B4,PR,I6
QC,#na#,BR5D,43512,BA,B1,BA,QC
QS,#na#,BR4S,38777,RJ,B1,RJ,QS
SH,K,BR2C,#na#,RJ,B4,RJ,SH


epoch,train_loss,valid_loss,accuracy,time


KeyboardInterrupt: 

In [9]:
model.save('.', 'teste')

# Load fitted pipeline instance

In [11]:
loaded_model = ClassificationPipeline.load('teste')
val_dict = loaded_model.validate(data = test_data)

C:\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
C:\Anaconda3\lib\site-packages\sklearn\metrics\_classification.py:1272: UndefinedMetricWarning: Recall and F-score are ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


# teste

In [12]:
val_dict['data_df'][['_TRUE_LABEL_PROBA', '_CLASS_PROBA','_GOT_RIGHT','_UNSEEN_CLASS']].mean()


_TRUE_LABEL_PROBA    0.000000
_CLASS_PROBA         0.037976
_GOT_RIGHT           0.237100
_UNSEEN_CLASS        0.430700
dtype: float64